In [ ]:
!pip install -r ./requirements.txt -q

In [ ]:
!pip show langchain

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

os.environ.get('PINECONE_API_KEY')

## LLM Models (Wrappers): GPT3

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7, max_tokens=512)
print(llm)

In [ ]:
output = llm('explain quantum mechanics in one sentance')
print (output)

In [ ]:
print (llm.get_num_tokens('explain quantum mechanics in one sentance'))

In [ ]:
output= llm.generate(['....is the capital of framce', "what is the formula for the area of a circle"])

In [ ]:
print (output.generations)

In [ ]:
print (output.generations[1][0].text)

In [ ]:
len(output.generations)

In [ ]:
foroutput = llm.generate(["write an original tagline for a burger resturatint"] * 3)

In [ ]:
for o in output.generations:
    print (o[0].text, end='')

#### ChatModels: GPT-3.5-Turbo and GPT-4

TODO: https://platform.openai.com/docs/guides/text-generation/chat-completions-api
Read more about what the different roles are

In [ ]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI


In [ ]:
chat = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content="you are a physicist and only respond in german"),
    HumanMessage(content="explain quantum mechanics in one sentance")
]
output = chat(messages)

In [ ]:
print(output.content)

### Prompt Templates

In [ ]:
from langchain import PromptTemplate

In [ ]:
template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

print (prompt)

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7)

output = llm(prompt.format(virus="ebola",language='english'))
print (output)

#### Simple Chains

In [ ]:
pip install openai==0.28.1  

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)

template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

chain = LLMChain(llm=llm,prompt=prompt)
output = chain.run({"virus": "HSV","language":"english"})

#  output = chain.run('HSV')

In [ ]:
print(output)

### Sequential Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name="text-davinci-003", temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''
    you are an experiance scientist and Python programmer.  Write a function the implements the concept of {concept}.
    '''
)
chain1 = LLMChain(llm=llm1,prompt=prompt1)

llm2 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='''
    given the python function {function}, describe it as detailed as possible. 
    '''
)
chain2 = LLMChain(llm=llm2,prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose=True)

output = overall_chain.run('linear regression')




### LangChain Agents

In [4]:
print ( 5.1 ** 7.3)
print (5.1 ** 1.7)

146306.05007233328
15.953989407385711


In [8]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [15]:
llm = OpenAI(temperature=0)
agent_executer = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
) 
# agent_executer.run("calculate the square root of the factorial of 20 and display it with four decimal points")

# agent_executer.run("what is the answer to 5.1 ** 7.3")

# agent_executer.run("what is the external ipdress of this box this query is running and where is it located, print it out with format (ip,city,country)")
# agent_executer.run("what are the current files in the directory")

# agent_executer.run("can you remove the file \"RM ME\" in the current directory")




> Entering new AgentExecutor chain...
 I need to use a command to list the files
Action: Python_REPL
Action Input: import os; print(os.listdir())
Observation: ['.env', '.git', '.gitignore', '01_langchain_basics.ipynb', 'README.md', 'requirements.txt', 'RM ME']

Thought: I now know the final answer
Final Answer: The current files in the directory are: .env, .git, .gitignore, 01_langchain_basics.ipynb, README.md, requirements.txt, and RM ME.

> Finished chain.


> Entering new AgentExecutor chain...
 I need to use a command to delete a file
Action: Python_REPL
Action Input: import os; os.remove("RM ME")
Observation: 
Thought: I should check if the file was removed
Action: Python_REPL
Action Input: import os; os.path.exists("RM ME")
Observation: 
Thought: I now know the final answer
Final Answer: False

> Finished chain.


'False'